In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
# Read the combined cells with access variables
gdf_cells_access = gpd.read_parquet("outputs/celdas_combined_access_v3.parquet")
gdf_cells_access_br = gpd.read_parquet("outputs/celdas_combined_bra_access_v3.parquet")

In [ ]:
gdf_cells_access.info()

In [ ]:
gdf_cells_access.head()

In [ ]:
gdf_cells_access_br.head()

In [ ]:
gdf_cells_access_br = gdf_cells_access_br.drop("index", axis=1)

In [ ]:
gdf_cells_access_br.columns

In [ ]:
gdf_cells_access.columns

In [ ]:
filter_cols = [
    # Cell variables
    "cell_id",
    "polygon_id",
    "smod",  # urbanization degree
    "category",  # urbanization category
    "code",  # country
    "lon",
    "lat",  # cell centroid
    "geometry",  # cell polygon
    # Accessibility to primary schools
    "nearest_primary_schools_ix",
    "distance_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools_label",
    "id_edificio",
    "lat_primary_school",
    "lon_primary_school",
    # Acessibility to middle schools
    "nearest_middle_schools_ix",
    "distance_to_nearest_middle_schools",
    "duration_to_nearest_middle_schools",
    "duration_to_nearest_middle_schools_label",
    "id_edificio_middle_school",
    "lat_middle_school",
    "lon_middle_school",
    # Acessibility to secondary schools
    "nearest_secondary_schools_ix",
    "distance_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools_label",
    "id_edificio_secondary_school",
    "lat_secondary_school",
    "lon_secondary_school",
]

In [ ]:
gdf_cells_access_concat = pd.concat(
    [gdf_cells_access[filter_cols], gdf_cells_access_br[filter_cols]], ignore_index=True
)

In [ ]:
gdf_cells_access_concat.columns

In [ ]:
# Read the combined cells with the worldpop variables
gdf_cells_pop = gpd.read_parquet("outputs/celdas_combined_pop.parquet")
gdf_cells_pop_bra = gpd.read_parquet(
    "outputs/brazil_worldpop_school_age_celdas_01_07_2025.parquet"
)

In [ ]:
gdf_cells_pop.info()

In [ ]:
gdf_cells_pop.columns

In [ ]:
gdf_cells_pop[
    [
        "pop_2020_m_5",
        "pop_2020_m_10",
        "pop_2020_m_15",
        "pop_2020_f_5",
        "pop_2020_f_10",
        "pop_2020_f_15",
    ]
]

In [ ]:
gdf_cells_pop.head()

In [ ]:
gdf_cells_pop_bra.head()

In [ ]:
gdf_cells_pop.columns

In [ ]:
gdf_cells_pop["polygon_id"]

In [ ]:
gdf_cells_pop_bra

In [ ]:
# gdf_cells_pop_bra["polygon_id"] = range(1000000, 1000000 + len(gdf_cells_pop_bra))

In [ ]:
gdf_cells_pop_bra = gdf_cells_pop_bra.rename({"code": "country"}, axis=1)

In [ ]:
gdf_cells_pop_bra.columns

In [ ]:
gdf_cells_pop_concat = pd.concat(
    [gdf_cells_pop[gdf_cells_pop_bra.columns], gdf_cells_pop_bra], ignore_index=True
)

### IMPORTANT

Date: 04 Jun 2025

We are splitting the population in secondary in middel and secondary.

### Population Split

Middle: Population between 10 and 15 yo  
Secondary: Population between 15 and 20 yo


In [ ]:
gdf_cells_pop_concat.columns

In [ ]:
gdf_cells_pop_concat["pop_secondary_school_age"] = gdf_cells_pop_concat[
    ["pop_2020_m_15", "pop_2020_f_10"]
].sum(axis=1)
gdf_cells_pop_concat["pop_middle_school_age"] = gdf_cells_pop_concat[
    ["pop_2020_m_10", "pop_2020_f_10"]
].sum(axis=1)

In [ ]:
index_col = ["cell_id"]
common_cols = ["smod", "polygon_id", "geometry", "country"]
access_cols = [
    "lat",
    "lon",
    "category",  # urbanization category (new in v3)
    "nearest_primary_schools_ix",
    "distance_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools",
    "duration_to_nearest_primary_schools_label",
    "id_edificio",
    "lat_primary_school",
    "lon_primary_school",
    "nearest_middle_schools_ix",
    "distance_to_nearest_middle_schools",
    "duration_to_nearest_middle_schools",
    "duration_to_nearest_middle_schools_label",
    "id_edificio_middle_school",
    "lat_middle_school",
    "lon_middle_school",
    "nearest_secondary_schools_ix",
    "distance_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools",
    "duration_to_nearest_secondary_schools_label",
    "id_edificio_secondary_school",
    "lat_secondary_school",
    "lon_secondary_school",
]
pop_cols = [
    "pop_2020_m_5",
    "pop_2020_f_5",
    "pop_2020_m_10",
    "pop_2020_f_10",
    "pop_2020_m_15",
    "pop_2020_f_15",
    "pop_m",
    "pop_f",
    "pop_total",
    "pop_primary_school_age",
    "pop_middle_school_age",
    "pop_secondary_school_age",
]

In [ ]:
gdf_cells_pop_concat

In [ ]:
gdf_cells_access_concat["polygon_id"].head()

In [ ]:
gdf_cells_access_concat["polygon_id"].tail()

In [ ]:
gdf_cells_pop_concat["polygon_id"].head()

In [ ]:
gdf_cells_pop_concat["polygon_id"].tail()

In [ ]:
gdf_cells_access_concat["cell_id"] = gdf_cells_access_concat["cell_id"].astype("int32")
gdf_cells_pop_concat["cell_id"] = gdf_cells_pop_concat["cell_id"].astype("int32")

In [ ]:
gdf_cells_access_concat

In [ ]:
# Combine the two datasets using the index_col
gdf_combined = gdf_cells_access_concat[index_col + access_cols].merge(
    gdf_cells_pop_concat[index_col + common_cols + pop_cols],
    on=index_col,
    suffixes=("_access", "_pop"),
)

# Display the combined dataset
gdf_combined.head()

In [ ]:
celdas_original = gpd.read_file(
    "inputs/Asentamientos humanos 2/Polígonos/Nuevos/Educación/CELDAS.gpkg"
)
celdas_original.shape

In [ ]:
len(celdas_original.polygon_id.unique())

In [ ]:
celdas_original["pop_2020"].sum()

In [ ]:
celdas_original.drop_duplicates(subset="polygon_id")["pop_2020"].sum()

In [ ]:
celdas_original.columns

In [ ]:
# 333 804 798 <--- celdas originales con polygon_id duplicados
#  38 028 451 <--- celdas originales sin polygon_id duplicados
#  48 359 084 <--- celdas corregidas con polygon_id duplicados
#   5 241 000 <--- celdas corregidas sin polygon_id duplicados
# 211.9M in 2020 en todo brazil segun el IBGE

In [ ]:
gdf_combined["polygon_id"] = gdf_combined["polygon_id"].astype(str)

In [ ]:
gdf_combined = gpd.GeoDataFrame(gdf_combined)

In [ ]:
gdf_combined.crs

In [ ]:
gdf_combined.to_parquet("outputs/celdas_pop_distance_complete_v3.parquet", index=False)

In [ ]:
gdf_combined.shape[0], gdf_cells_access_concat.shape[0], gdf_cells_pop_concat.shape[0]

In [ ]:
gdf_combined.columns

In [ ]:
gdf_combined = gpd.GeoDataFrame(
    gdf_combined, geometry=gdf_combined.geometry, crs=gdf_cells_access.crs
)

In [ ]:
gdf_combined["polygon_id"] = gdf_combined["polygon_id"].astype(str)

In [ ]:
gdf_combined.to_file(
    "outputs/celdas_pop_distance_complete_v3.geojson", driver="GeoJSON", index=False
)

In [ ]:
gdf_combined.to_parquet("outputs/celdas_pop_distance_complete_v3.parquet", index=False)

In [ ]:
gdf_combined_stats = gdf_combined[
    [
        "cell_id",
        "polygon_id",
        "category",  # urbanization category
        "country",
        "pop_total",
        "pop_primary_school_age",
        "pop_middle_school_age",
        "pop_secondary_school_age",
        "distance_to_nearest_primary_schools",
        "duration_to_nearest_primary_schools",
        "duration_to_nearest_primary_schools_label",
        "distance_to_nearest_middle_schools",
        "duration_to_nearest_middle_schools",
        "duration_to_nearest_middle_schools_label",
        "distance_to_nearest_secondary_schools",
        "duration_to_nearest_secondary_schools",
        "duration_to_nearest_secondary_schools_label",
    ]
]

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(gdf_combined_stats, title="Data Report")

In [ ]:
profile.to_file("outputs/data_report.html")

In [ ]:
gdf_combined_stats.head().to_clipboard()

In [ ]:
# Add "No access" as a category for primary schools
if (
    "No access"
    not in gdf_combined_stats[
        "duration_to_nearest_primary_schools_label"
    ].cat.categories
):
    gdf_combined_stats["duration_to_nearest_primary_schools_label"] = (
        gdf_combined_stats[
            "duration_to_nearest_primary_schools_label"
        ].cat.add_categories("No access")
    )

# Fill missing values with "No access" for primary schools
gdf_combined_stats["duration_to_nearest_primary_schools_label"] = gdf_combined_stats[
    "duration_to_nearest_primary_schools_label"
].fillna("No access")

# Add "No access" as a category for middle schools
if (
    "No access"
    not in gdf_combined_stats["duration_to_nearest_middle_schools_label"].cat.categories
):
    gdf_combined_stats["duration_to_nearest_middle_schools_label"] = gdf_combined_stats[
        "duration_to_nearest_middle_schools_label"
    ].cat.add_categories("No access")

# Fill missing values with "No access" for middle schools
gdf_combined_stats["duration_to_nearest_middle_schools_label"] = gdf_combined_stats[
    "duration_to_nearest_middle_schools_label"
].fillna("No access")


# Add "No access" as a category for secondary schools
if (
    "No access"
    not in gdf_combined_stats[
        "duration_to_nearest_secondary_schools_label"
    ].cat.categories
):
    gdf_combined_stats["duration_to_nearest_secondary_schools_label"] = (
        gdf_combined_stats[
            "duration_to_nearest_secondary_schools_label"
        ].cat.add_categories("No access")
    )

# Fill missing values with "No access" for secondary schools
gdf_combined_stats["duration_to_nearest_secondary_schools_label"] = gdf_combined_stats[
    "duration_to_nearest_secondary_schools_label"
].fillna("No access")

In [ ]:
gdf_combined_stats.shape, gdf_combined.shape

In [ ]:
gdf_combined_stats.loc[:, "geometry"] = gdf_combined["geometry"].values

In [ ]:
gdf_combined_stats = gpd.GeoDataFrame(gdf_combined_stats)

In [ ]:
gdf_combined_stats.to_parquet(
    "outputs/gdf_celdas_final_2025_07_01.parquet", index=False
)

In [ ]:
# Group by "country", "smod", and "duration_to_nearest_primary_schools_label" and calculate the sum of "pop_primary_school_age"
result = gdf_combined_stats.groupby(
    ["country", "category", "duration_to_nearest_primary_schools_label"], as_index=False
)["pop_primary_school_age"].sum()

# Display the result
result

In [ ]:
# Save to an excel file
result.to_excel("outputs/pop_primary_school_age_2025_06_04.xlsx", index=False)

In [ ]:
# Group by "country", "category", and "duration_to_nearest_schools_label" and calculate the sum of "pop_middle_school_age"
result_middle = gdf_combined_stats.groupby(
    ["country", "category", "duration_to_nearest_middle_schools_label"],
    as_index=False,
)["pop_middle_school_age"].sum()

# Display the result
result_middle

In [ ]:
# Save to an excel file
result_middle.to_excel("outputs/pop_middle_school_age_2025_06_17.xlsx", index=False)

In [ ]:
# Group by "country", "category", and "duration_to_nearest_schools_label" and calculate the sum of "pop_secondary_school_age"
result_secondary = gdf_combined_stats.groupby(
    ["country", "category", "duration_to_nearest_secondary_schools_label"],
    as_index=False,
)["pop_secondary_school_age"].sum()

# Display the result
result_secondary

In [ ]:
result.head().to_clipboard()

In [ ]:
# Import required libraries
import pandas as pd
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
# Create a bar plot with seaborn
plt.figure(figsize=(8, 6))
sns.barplot(
    data=result_secondary,
    x="duration_to_nearest_secondary_schools_label",
    y="pop_secondary_school_age",
    hue="category",
    errorbar=None,
)
plt.title("Population in Secondary School Age by Travel Time")
plt.xlabel("Travel Time to Nearest School (minutes)")
plt.ylabel("Population (Secondary School Age)")
plt.tight_layout()
plt.show()

In [ ]:
result_secondary

In [ ]:
for col in result_secondary.country.unique():
    # Create a bar plot with seaborn
    plt.figure(figsize=(8, 6))
    sns.barplot(
        data=result_secondary[result_secondary["country"] == col],
        x="duration_to_nearest_secondary_schools_label",
        y="pop_secondary_school_age",
        hue="category",
        errorbar=None,
    )
    plt.title(f"{col}: Population in Secondary School Age by Travel Time")
    plt.xlabel("Travel Time to Nearest School (minutes)")
    plt.ylabel("Population (Secondary School Age)")
    plt.tight_layout()
    plt.savefig(f"outputs/{col}_secondary_school_access_plot.png")
    plt.show()

In [ ]:
result

In [ ]:
# (optional) set a clean style
sns.set(style="whitegrid", context="talk")

In [ ]:
# make a working df
df = gdf_combined_stats.copy()

plt.figure(figsize=(8, 6))
sns.scatterplot(
    x="duration_to_nearest_primary_schools",
    y="pop_primary_school_age",
    hue="country",
    palette="tab10",
    alpha=0.1,
    data=df,
)
plt.xlabel("Travel time to nearest primary school (min)")
plt.ylabel("Primary - school - age population")
plt.title("Primary - age children vs. travel time to primary school")
plt.legend(title="Country", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x="duration_to_nearest_secondary_schools",
    y="pop_secondary_school_age",
    hue="country",
    palette="tab10",
    alpha=0.1,
    data=df,
)
plt.xlabel("Travel time to nearest secondary school (min)")
plt.ylabel("Secondary - school - age population")
plt.title("Secondary - age children vs. travel time to secondary school")
plt.legend(title="Country", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# aggregate by country
country_stats = (
    df.groupby("country")
    .agg(
        total_primary_pop=("pop_primary_school_age", "sum"),
        mean_primary_time=("duration_to_nearest_primary_schools", "mean"),
        total_secondary_pop=("pop_secondary_school_age", "sum"),
        mean_secondary_time=("duration_to_nearest_secondary_schools", "mean"),
    )
    .reset_index()
)

fig, ax = plt.subplots(2, 1, figsize=(10, 12), sharex=True, sharey=True)

# primary
sns.scatterplot(
    x="mean_primary_time",
    y="total_primary_pop",
    size="total_primary_pop",
    sizes=(100, 2000),
    hue="country",
    data=country_stats,
    legend=False,
    alpha=0.5,
    ax=ax[0],
)
for _, row in country_stats.iterrows():
    ax[0].text(
        row.mean_primary_time,
        row.total_primary_pop,
        row.country.upper(),
        horizontalalignment="center",
        verticalalignment="center",
    )
ax[0].set_xlabel("Avg. primary - school travel time (min)")
ax[0].set_ylabel("Total primary - age population")
ax[0].set_title("Country - level: primary - age pop vs. avg. travel time")


# primary
sns.scatterplot(
    x="mean_secondary_time",
    y="total_secondary_pop",
    size="total_secondary_pop",
    sizes=(100, 2000),
    hue="country",
    data=country_stats,
    legend=False,
    alpha=0.5,
    ax=ax[1],
)
for _, row in country_stats.iterrows():
    ax[1].text(
        row.mean_secondary_time,
        row.total_secondary_pop,
        row.country.upper(),
        horizontalalignment="center",
        verticalalignment="center",
    )
ax[1].set_xlabel("Avg. secondary - school travel time (min)")
ax[1].set_ylabel("Total secondary - age population")
ax[1].set_title("Country - level: secondary - age pop vs. avg. travel time")

plt.tight_layout()
plt.show()

In [ ]:
category_stats = (
    df.groupby("category")
    .agg(
        total_primary_pop=("pop_primary_school_age", "sum"),
        mean_primary_time=("duration_to_nearest_primary_schools", "mean"),
        total_secondary_pop=("pop_secondary_school_age", "sum"),
        mean_secondary_time=("duration_to_nearest_secondary_schools", "mean"),
    )
    .sort_values("mean_primary_time", ascending=False)
    .reset_index()
)

# Primary
fig, ax = plt.subplots(1, 2, figsize=(14, 6), sharex=True)
sns.barplot(
    x="mean_primary_time",
    y="category",
    data=category_stats,
    ax=ax[0],
    order=category_stats.category,
)
ax[0].set_title("Avg. primary travel time by settlement type")
ax[0].set_xlabel("Time (min)")
ax[0].set_ylabel("Settlement type")

# Secondary
sns.barplot(
    x="mean_secondary_time",
    y="category",
    data=category_stats.sort_values("mean_secondary_time", ascending=False),
    ax=ax[1],
)
ax[1].set_title("Avg. secondary travel time by settlement type")
ax[1].set_xlabel("Time (min)")
ax[1].set_ylabel("")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

In [ ]:
# Define duration bin order
duration_order = (
    gdf_combined_stats["duration_to_nearest_primary_schools_label"].dropna().unique()
)

In [ ]:
duration_order.to_list()

In [ ]:
duration_order = [
    "0-15",
    "15-30",
    "30-45",
    "45-60",
    "60-90",
    "90-120",
    ">120",
    "No access",
]

In [ ]:
cmap = cm.get_cmap("viridis", len(duration_order))

In [ ]:
duration_colors = {
    label: mcolors.rgb2hex(cmap(i)) for i, label in enumerate(duration_order)
}
duration_colors

In [ ]:
duration_colors["No access"] = "#808080"  # Set "No access" to gray

In [ ]:
def plot_school_age_distribution(
    education_level, hue, pop_label_df, duration_order, duration_colors, **kwargs
):
    """
    Plots the country-level distribution of school-age population by travel-time label.

    Parameters:
        education_level (str): The education level to plot ('primary', 'middle', or 'secondary').
        pop_label_df (pd.DataFrame): DataFrame with countries as index and travel-time bins as columns (absolute values).
        duration_order (list): List of travel-time bin labels in desired order.
        duration_colors (dict): Mapping from travel-time bin label to color.
    """

    # pivot to get total secondary‐age pop by country × label
    pop_label_country_sec = (
        pop_label_df.groupby(
            [hue, f"duration_to_nearest_{education_level}_schools_label"]
        )[f"pop_{education_level}_school_age"]
        .sum()
        .unstack(fill_value=0)
    )

    # Convert to fractions (so bars sum to 1)
    pop_label_pct = pop_label_country_sec.div(pop_label_country_sec.sum(axis=1), axis=0)

    # Plot
    ax = pop_label_pct[duration_order].plot(
        kind="bar",
        stacked=True,
        figsize=(10, 6),
        color=[duration_colors[label] for label in duration_order],
        width=0.8,
        linewidth=0,
        **kwargs,
    )
    ax.set_ylabel(f"Percentage of {education_level.capitalize()} population")
    ax.set_xlabel(hue.capitalize())
    ax.set_title(f"{education_level.capitalize()} age population by travel time bins")
    ax.legend(title="Travel time (min)", bbox_to_anchor=(1.05, 1), loc="upper left")

    # Invert the order of the legend to match the order of the bars
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(
        handles[::-1],
        labels[::-1],
        title="Travel time (min)",
        bbox_to_anchor=(1.05, 1),
        loc="upper left",
    )

    plt.tight_layout()

In [ ]:
plot_school_age_distribution(
    "primary", "country", gdf_combined_stats, duration_order, duration_colors
)
plt.savefig("outputs/figures/school_age_distribution_primary_country", dpi=300)
plt.show()
plot_school_age_distribution(
    "middle", "country", gdf_combined_stats, duration_order, duration_colors
)
plt.savefig("outputs/figures/school_age_distribution_middle_country", dpi=300)
plt.show()
plot_school_age_distribution(
    "secondary", "country", gdf_combined_stats, duration_order, duration_colors
)
plt.savefig("outputs/figures/school_age_distribution_secondary_country", dpi=300)
plt.show()

In [ ]:
plot_school_age_distribution(
    "primary",
    "category",
    gdf_combined_stats,
    duration_order,
    duration_colors,
    rot=0,
)
plt.savefig("outputs/figures/school_age_distribution_primary_urban", dpi=300)
plt.show()
plot_school_age_distribution(
    "middle",
    "category",
    gdf_combined_stats,
    duration_order,
    duration_colors,
    rot=0,
)
plt.savefig("outputs/figures/school_age_distribution_middle_urban", dpi=300)
plt.show()
plot_school_age_distribution(
    "secondary",
    "category",
    gdf_combined_stats,
    duration_order,
    duration_colors,
    rot=0,
)
plt.savefig("outputs/figures/school_age_distribution_secondary_urban", dpi=300)
plt.show()

In [ ]:
# Create a copy with relevant columns
df_lorenz = (
    gdf_combined_stats[
        ["duration_to_nearest_primary_schools", "pop_primary_school_age"]
    ]
    .dropna()
    .copy()
)

# Remove zero-pop cells
df_lorenz = df_lorenz[df_lorenz.pop_primary_school_age > 0]

# Sort by duration
df_lorenz.sort_values("duration_to_nearest_primary_schools", inplace=True)

# Compute cumulative population and cumulative share
df_lorenz["cum_pop"] = df_lorenz["pop_primary_school_age"].cumsum()
df_lorenz["cum_pop_share"] = (
    df_lorenz["cum_pop"] / df_lorenz["pop_primary_school_age"].sum()
)

# Weight durations by population (population-weighted Lorenz curve)
df_lorenz["duration_weighted"] = (
    df_lorenz["duration_to_nearest_primary_schools"]
    * df_lorenz["pop_primary_school_age"]
)
df_lorenz["cum_duration"] = df_lorenz["duration_weighted"].cumsum()
df_lorenz["cum_duration_share"] = (
    df_lorenz["cum_duration"] / df_lorenz["duration_weighted"].sum()
)

# Create a copy with relevant columns
df_lorenz_mid = (
    gdf_combined_stats[["duration_to_nearest_middle_schools", "pop_middle_school_age"]]
    .dropna()
    .copy()
)

# Remove zero-pop cells
df_lorenz_mid = df_lorenz_mid[df_lorenz_mid.pop_middle_school_age > 0]

# Sort by duration
df_lorenz_mid.sort_values("duration_to_nearest_middle_schools", inplace=True)

# Compute cumulative population and cumulative share
df_lorenz_mid["cum_pop"] = df_lorenz_mid["pop_middle_school_age"].cumsum()
df_lorenz_mid["cum_pop_share"] = (
    df_lorenz_mid["cum_pop"] / df_lorenz_mid["pop_middle_school_age"].sum()
)

# Weight durations by population (population-weighted Lorenz curve)
df_lorenz_mid["duration_weighted"] = (
    df_lorenz_mid["duration_to_nearest_middle_schools"]
    * df_lorenz_mid["pop_middle_school_age"]
)
df_lorenz_mid["cum_duration"] = df_lorenz_mid["duration_weighted"].cumsum()
df_lorenz_mid["cum_duration_share"] = (
    df_lorenz_mid["cum_duration"] / df_lorenz_mid["duration_weighted"].sum()
)

# Create a copy with relevant columns
df_lorenz_sec = (
    gdf_combined_stats[
        ["duration_to_nearest_secondary_schools", "pop_secondary_school_age"]
    ]
    .dropna()
    .copy()
)

# Remove zero-pop cells
df_lorenz_sec = df_lorenz_sec[df_lorenz_sec.pop_secondary_school_age > 0]

# Sort by duration
df_lorenz_sec.sort_values("duration_to_nearest_secondary_schools", inplace=True)

# Compute cumulative population and cumulative share
df_lorenz_sec["cum_pop"] = df_lorenz_sec["pop_secondary_school_age"].cumsum()
df_lorenz_sec["cum_pop_share"] = (
    df_lorenz_sec["cum_pop"] / df_lorenz_sec["pop_secondary_school_age"].sum()
)

# Weight durations by population (population-weighted Lorenz curve)
df_lorenz_sec["duration_weighted"] = (
    df_lorenz_sec["duration_to_nearest_secondary_schools"]
    * df_lorenz_sec["pop_secondary_school_age"]
)
df_lorenz_sec["cum_duration"] = df_lorenz_sec["duration_weighted"].cumsum()
df_lorenz_sec["cum_duration_share"] = (
    df_lorenz_sec["cum_duration"] / df_lorenz_sec["duration_weighted"].sum()
)

# Plot both Lorenz curves on the same plot
plt.figure(figsize=(8, 6))
plt.plot(
    df_lorenz["cum_pop_share"],
    df_lorenz["cum_duration_share"],
    label="Primary school age",
    color="blue",
    alpha=0.5,
)
plt.plot(
    df_lorenz_mid["cum_pop_share"],
    df_lorenz_mid["cum_duration_share"],
    label="Middle school age",
    color="orange",
    alpha=0.5,
)
plt.plot(
    df_lorenz_sec["cum_pop_share"],
    df_lorenz_sec["cum_duration_share"],
    label="Secondary school age",
    color="green",
    alpha=0.5,
)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Line of equality")
plt.title("Lorenz Curve: Inequality in Access to Schools")
plt.xlabel("Cumulative share of population")
plt.ylabel("Cumulative share of total travel-time burden")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig("outputs/figures/access_inequality_lorenz_curve.png", dpi=300)

plt.show()

In [ ]:
df_lorenz[df_lorenz["cum_pop_share"] >= 0.8][["cum_pop_share", "cum_duration_share"]]

In [ ]:
df_lorenz_sec[df_lorenz_sec["cum_pop_share"] >= 0.8][
    ["cum_pop_share", "cum_duration_share"]
]

In [ ]:
df_lorenz_sec[["cum_pop_share", "cum_duration_share"]]

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_primary_school_age"] > 0],
    x="country",
    y="duration_to_nearest_primary_schools",
    color="skyblue",
    showfliers=False,
    width=0.4,
    # position=1,
    boxprops=dict(alpha=0.7),
    linewidth=1.5,
    dodge=True,
    label="Primary",
)
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_middle_school_age"] > 0],
    x="country",
    y="duration_to_nearest_middle_schools",
    color="skyblue",
    showfliers=False,
    width=0.4,
    # position=1,
    boxprops=dict(alpha=0.7),
    linewidth=1.5,
    dodge=True,
    label="Middle",
)
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_secondary_school_age"] > 0],
    x="country",
    y="duration_to_nearest_secondary_schools",
    color="violet",
    showfliers=False,
    width=0.4,
    # position=0,
    boxprops=dict(alpha=0.7),
    linewidth=1.5,
    dodge=True,
    label="Secondary",
)
plt.ylabel("Travel time to school (min)")
plt.title("Distribution of Travel Times to Schools by Country")
plt.legend(
    handles=[
        plt.Line2D([0], [0], color="skyblue", lw=8, label="Primary"),
        plt.Line2D([0], [0], color="violet", lw=8, label="Middle"),
        plt.Line2D([0], [0], color="orange", lw=8, label="Secondary"),
    ]
)
plt.tight_layout()
plt.show()

In [ ]:
# Summary stats for Ecuador
ecuador = gdf_combined_stats[gdf_combined_stats["country"] == "ecu"]
print("Number of rows for Ecuador:", len(ecuador))
print("Non-zero population cells:", (ecuador["pop_primary_school_age"] > 0).sum())
print(
    "Non-null durations:",
    ecuador["duration_to_nearest_primary_schools"].notnull().sum(),
)
print(
    "Unique duration values:", ecuador["duration_to_nearest_primary_schools"].unique()
)

In [ ]:
gdf_combined_stats[gdf_combined_stats["country"] == "ecu"][
    ["pop_primary_school_age", "duration_to_nearest_primary_schools"]
].describe()

In [ ]:
sns.boxplot(
    data=gdf_combined_stats[
        (gdf_combined_stats["pop_primary_school_age"] > 0)
        & (gdf_combined_stats["duration_to_nearest_primary_schools"] > 0)
    ],
    x="country",
    y="duration_to_nearest_primary_schools",
    showfliers=False,  # hide outliers for readability,
)
plt.ylabel("Travel time to primary school (min)")
plt.title("Travel Times to Primary Schools by Country")
plt.tight_layout()
plt.show()

In [ ]:
gdf_combined_stats[gdf_combined_stats["pop_primary_school_age"] > 0]

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=gdf_combined_stats[gdf_combined_stats["pop_primary_school_age"] > 0],
    x="category",
    y="duration_to_nearest_primary_schools",
    order=["non_urban_area", "urban_area"],
    showfliers=False,
)
plt.ylabel("Travel time to primary school (min)")
plt.title("Travel Time Distribution by Settlement Type")
plt.tight_layout()
plt.show()

In [ ]:
# Make sure all needed columns are included in the copy
df_facet = gdf_combined_stats[
    [
        "country",
        "category",
        "pop_primary_school_age",
        "pop_middle_school_age",
        "pop_secondary_school_age",
        "duration_to_nearest_primary_schools_label",
        "duration_to_nearest_middle_schools_label",
        "duration_to_nearest_secondary_schools_label",
    ]
].copy()

# Melt the population columns
df_melted = pd.melt(
    df_facet,
    id_vars=[
        "country",
        "category",
        "duration_to_nearest_primary_schools_label",
        "duration_to_nearest_middle_schools_label",
        "duration_to_nearest_secondary_schools_label",
    ],
    value_vars=[
        "pop_primary_school_age",
        "pop_middle_school_age",
        "pop_secondary_school_age",
    ],
    var_name="school_level",
    value_name="pop_school_age",
)

# Assign matching duration labels
df_melted["duration_label"] = df_melted.apply(
    lambda row: (
        row["duration_to_nearest_primary_schools_label"]
        if row["school_level"] == "pop_primary_school_age"
        else (
            row["duration_to_nearest_middle_schools_label"]
            if row["school_level"] == "pop_middle_school_age"
            else row["duration_to_nearest_secondary_schools_label"]
        )
    ),
    axis=1,
)

# Clean up school level name
df_melted["school_level"] = df_melted["school_level"].map(
    {
        "pop_primary_school_age": "Primary",
        "pop_middle_school_age": "Middle",
        "pop_secondary_school_age": "Secondary",
    }
)

# Filter out zero-pop rows
df_melted = df_melted[df_melted["pop_school_age"] > 0]

# Facet by country
g = sns.catplot(
    data=df_melted,
    kind="bar",
    x="duration_label",
    y="pop_school_age",
    hue="school_level",
    hue_order=["Primary", "Middle", "Secondary"],
    col="country",
    col_wrap=3,
    order=duration_order,
    height=10,
    aspect=0.7,
    errorbar=None,
    sharex=False,
    sharey=True,
)

g.set_axis_labels("Travel time bin (min)", "School-age population")
# g.set_titles("Country: {col_name}")
g._legend.set_title("School level")
g._legend.set_loc("lower right")
# Rotate x-axis labels for better readability
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)
        label.set_ha("right")
# plt.subplots_adjust(top=0.9)
plt.tight_layout()

plt.savefig("outputs/figures/school_population_duration_country.png", dpi=300)
plt.show()

In [ ]:
gdf_amazon = gpd.read_parquet(
    "~/Documents/amazon_fluvial_transport_net/osm_amazonia_streets.parquet"
)

In [ ]:
gdf_amazon.crs

In [ ]:
gdf_amazon.head()

In [ ]:
gdf_amazon.columns

In [ ]:
gdf_amazon.geometry.type.value_counts()

In [ ]:
gdf_amazon.fclass.value_counts()

In [ ]:
gdf_amazon[gdf_amazon.fclass.isin(["boat", "ferry"])].shape

In [ ]:
# Define bounding box around Iquitos
lon_min, lon_max = -73.5, -73.0
lat_min, lat_max = -4.0, -3.5

gdf_amazon_iquitos = gdf_amazon[
    (gdf_amazon.geometry.bounds["minx"] >= lon_min)
    & (gdf_amazon.geometry.bounds["maxx"] <= lon_max)
    & (gdf_amazon.geometry.bounds["miny"] >= lat_min)
    & (gdf_amazon.geometry.bounds["maxy"] <= lat_max)
]

In [ ]:
gdf_amazon_iquitos.shape

In [ ]:
gdf_amazon_iquitos[gdf_amazon_iquitos.fclass.isin(["boat", "ferry"])].explore()

In [ ]:
gdf_combined_stats["polygon_id"].value_counts()

In [ ]:
gdf_combined_stats["polygon_id"].hist()

In [ ]:
gdf_combined_stats["polygon_id"].value_counts().head(50).plot(kind="bar")

In [ ]:
gdf_combined_stats["polygon_id"].value_counts().head(100)

In [ ]:
gdf_combined_stats["polygon_id"]

In [ ]:
gdf_combined_stats.columns

In [ ]:
gdf_combined_stats[gdf_combined_stats["polygon_id"] == "01817-1-2"].explore(
    column="pop_total"
)

In [ ]:
gdf_combined_stats[gdf_combined_stats["polygon_id"] == "01817-1-2"].explore(
    column="duration_to_nearest_primary_schools_label", cmap="viridis"
)

In [ ]:
gdf_combined_stats[gdf_combined_stats["polygon_id"] == "01817-1-2"].explore(
    column="duration_to_nearest_primary_schools_label", cmap="viridis"
)

In [ ]:
gdf_combined_stats.groupby("polygon_id")["pop_total"].sum().sort_values(
    ascending=False
).head(10).astype(int)

In [ ]:
gdf_combined_stats[gdf_combined_stats["polygon_id"] == "01817-1-2"]["pop_total"]

In [ ]:
# Group by polygon_id and travel time category, sum population for each education level

# For primary school age
primary_grouped = (
    gdf_combined_stats.groupby(
        ["polygon_id", "duration_to_nearest_primary_schools_label"]
    )["pop_primary_school_age"]
    .sum()
    .unstack(fill_value=0)
)
primary_grouped

In [ ]:
primary_grouped.head().to_clipboard()

In [ ]:
# For primary school age: polygons with at least 20% of their population within 30 minutes of a primary school

# Sum population in 0-15 and 15-30 min bins
within_30 = primary_grouped.get("0-15", 0) + primary_grouped.get("15-30", 0)
total_pop = primary_grouped.sum(axis=1)
share_within_30 = within_30 / total_pop.replace(
    0, float("nan")
)  # avoid division by zero

# Count polygons where at least 20% of population is within 30 minutes
num_polygons_20pct_within_30 = (share_within_30 >= 0.2).sum()
print(
    f"Number of polygons with at least 20% of their primary school age population within 30 minutes of a primary school: {num_polygons_20pct_within_30}"
)

In [ ]:
primary_grouped[">30"] = primary_grouped[
    ["30-45", "45-60", "60-90", "90-120", ">120"]
].sum(axis=1)

plus_30_min = primary_grouped[">30"].copy()
plus_30_min = plus_30_min.sort_values(ascending=False)
cum_pop_over_30 = plus_30_min.sort_values(ascending=False).cumsum()
plus_30_min_and_1_person = plus_30_min[plus_30_min > 1]
n_settlements = plus_30_min_and_1_person.shape[0]
n_people = plus_30_min_and_1_person.sum().round(0)
print(
    f"""{n_people:,.0f} niñas y niños en edad escolar primaria, 
que viven en {n_settlements:,} asentamientos humanos de la región amazónica, 
se encuentran a más de 30 minutos a pie de la escuela primaria más cercana."""
)

plt.figure(figsize=(10, 6))
cum_pop_over_30.plot()
plt.title("Cumulative Population with >30 min Travel Time to Primary School")
plt.xlabel("Settlement (sorted by population with >30 min travel time)")
plt.ylabel("Cumulative Population")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# For middle school age
middle_grouped = (
    gdf_combined_stats.groupby(
        ["polygon_id", "duration_to_nearest_middle_schools_label"]
    )["pop_middle_school_age"]
    .sum()
    .unstack(fill_value=0)
)
middle_grouped

In [ ]:
middle_grouped[">30"] = middle_grouped[
    ["30-45", "45-60", "60-90", "90-120", ">120"]
].sum(axis=1)

plus_30_min = middle_grouped[">30"].copy()
plus_30_min = plus_30_min.sort_values(ascending=False)
cum_pop_over_30 = plus_30_min.sort_values(ascending=False).cumsum()
plus_30_min_and_1_person = plus_30_min[plus_30_min > 1]
n_settlements = plus_30_min_and_1_person.shape[0]
n_people = plus_30_min_and_1_person.sum().round(0)
nivel_escolar = "secundaria baja"
print(
    f"""{n_people:,.0f} niñas y niños en edad escolar {nivel_escolar}, 
que viven en {n_settlements:,} asentamientos humanos de la región amazónica, 
se encuentran a más de 30 minutos a pie de la escuela {nivel_escolar} más cercana."""
)

plt.figure(figsize=(10, 6))
cum_pop_over_30.plot()
plt.title(f"Población acumulada con >30 min de viaje a la escuela {nivel_escolar}")
plt.xlabel("Asentamientos humanos (ordenados por población con >30 min de viaje)")
plt.ylabel("Población acumulada")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# For secondary school age
secondary_grouped = (
    gdf_combined_stats.groupby(
        ["polygon_id", "duration_to_nearest_secondary_schools_label"]
    )["pop_secondary_school_age"]
    .sum()
    .unstack(fill_value=0)
)
secondary_grouped

In [ ]:
secondary_grouped[">30"] = secondary_grouped[
    ["30-45", "45-60", "60-90", "90-120", ">120"]
].sum(axis=1)

plus_30_min = secondary_grouped[">30"].copy()
plus_30_min = plus_30_min.sort_values(ascending=False)
cum_pop_over_30 = plus_30_min.sort_values(ascending=False).cumsum()
plus_30_min_and_1_person = plus_30_min[plus_30_min > 1]
n_settlements = plus_30_min_and_1_person.shape[0]
n_people = plus_30_min_and_1_person.sum().round(0)
nivel_escolar = "secundaria alta"
print(
    f"""{n_people:,.0f} niñas y niños en edad escolar {nivel_escolar}, 
que viven en {n_settlements:,} asentamientos humanos de la región amazónica, 
se encuentran a más de 30 minutos a pie de la escuela {nivel_escolar} más cercana."""
)

plt.figure(figsize=(10, 6))
cum_pop_over_30.plot()
plt.title(f"Población acumulada con >30 min de viaje a la escuela {nivel_escolar}")
plt.xlabel("Asentamientos humanos (ordenados por población con >30 min de viaje)")
plt.ylabel("Población acumulada")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# Plot cumulative population lines as percentages
for grouped, color, label in [
    (primary_grouped, "blue", "Primaria"),
    (middle_grouped, "orange", "Sec Baja"),
    (secondary_grouped, "green", "Sec Alta"),
]:
    cum_pop = grouped[">30"].sort_values(ascending=False).cumsum().values
    cum_pop = np.insert(cum_pop, 0, 0)  # Add zero at the start
    total_pop = cum_pop[-1]
    x_vals = np.arange(len(cum_pop))
    plt.plot(
        x_vals,
        cum_pop / total_pop * 100,
        label=label,
        color=color,
    )

    # Add vertical lines at 80% cumulative population for each group
    idx_80 = (cum_pop >= 0.8 * total_pop).argmax()
    print(f"{idx_80=}")
    ymax = (cum_pop[idx_80] / total_pop) * 100

    plt.axvline(
        idx_80,
        ymin=0,
        ymax=80 / 105,
        color=color,
        linestyle="--",
        alpha=0.7,
        linewidth=1,
    )
    plt.text(
        idx_80,
        0,
        f"{idx_80:.0f}",
        color=color,
        va="top",
        ha="left",
        fontsize=10,
        fontdict={"weight": "bold"},
        rotation=90,
    )

plt.axhline(80, xmin=0, xmax=1, color="red", linestyle="--", alpha=0.7, linewidth=1)
plt.text(
    2000,  # x_vals[len(x_vals) // 2],
    80,
    f"{80:.0f}% de la población",
    color="red",
    va="bottom",
    ha="left",
    fontsize=10,
    rotation=0,
)

plt.ylim(0, 105)
plt.xlim(-50, len(cum_pop) - 1)

plt.xlabel("Cantidad de asentamientos humanos")
plt.ylabel("Población acumulada (%)")
plt.title("Población acumulada con >30 min de viaje a la escuela por nivel educativo")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(
    "outputs/figures/cumulative_population_over_30_min_travel_time.png", dpi=300
)
plt.show()

In [ ]:
polygon_ids_80pct

In [ ]:
gdf_combined_stats["80pct_color_primary"] = "gray"
gdf_combined_stats.loc[
    gdf_combined_stats["polygon_id"].isin(polygon_ids_80pct), "80pct_color_primary"
] = "red"

In [ ]:
grouped[">30"].sort_values(ascending=False).cumsum()

In [ ]:
gdf_combined_stats_polygon = gdf_combined_stats.dissolve(by="polygon_id")

In [ ]:
gdf_combined_stats_polygon.explore(
    color=gdf_combined_stats_polygon["80pct_color_Primaria"],
)

In [ ]:
gdf_combined_stats[gdf_combined_stats["polygon_id"] == "00004-1-1"][
    "pop_primary_school_age"
]

In [ ]:
gdf_combined_stats_polygon["pop_primary_school_age"]

In [ ]:
gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"].sum().loc[
    ["00006-1-1", "00004-1-1"]
]

In [ ]:
gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"].sum().loc[
    base_group.index
].min()

In [ ]:
gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"].sum().loc[
    base_group.index
].max()

In [ ]:
(
    gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"]
    .sum()
    .loc[base_group.index]
    / gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"]
    .sum()
    .loc[base_group.index]
    .max()
).min()

In [ ]:
(
    gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"]
    .sum()
    .loc[base_group.index]
    / gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"]
    .sum()
    .loc[base_group.index]
    .max()
).max()

In [ ]:
gdf_combined_stats_polygon.index.isin(["00004-1-1"])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# Plot countries with soft color palette and nice borders
# Plot all countries in the Amazon with normal style
countries.clip(amazon).plot(
    ax=ax,
    column="ADM0_PCODE",
    # cmap="tab20b",
    # alpha=0.6,
    facecolor="none",
    edgecolor="k",
    linewidth=1.5,
    legend=False,
    zorder=2,
)
if not countries_no_data.empty:
    countries_no_data.plot(
        ax=ax,
        facecolor="none",
        edgecolor="k",
        linewidth=1.5,
        hatch="///",
        zorder=2,
    )

# Add crosshatch to Venezuela, Guyana, and Suriname to indicate no data
no_data_countries = ["VE", "GY", "SR"]
countries_no_data = countries[countries["ADM0_PCODE"].isin(no_data_countries)].clip(
    amazon
)

# Plot Amazon boundary with a bold red edge and no fill
amazon.boundary.plot(
    ax=ax, color="red", linewidth=1.5, zorder=3, label="Amazonia Boundary"
)


amazon_bounds = amazon.buffer(1).total_bounds
amazon_bbox_poly = gpd.GeoSeries(box(*amazon_bounds))
amazon_outer = amazon_bbox_poly.difference(amazon)
amazon_outer.plot(
    ax=ax, facecolor="k", edgecolor="none", linewidth=0, zorder=2, alpha=0.5
)

## DATA HERE ##

# Calculate cumulative population for each group
total_pop = primary_grouped[">30"].sum()
print(f"{total_pop.sum()=}")
cum_pop = primary_grouped[">30"].sort_values(ascending=False).cumsum()
idx_80 = (cum_pop.values >= 0.8 * total_pop).argmax()

# Get the polygon_ids that account for 80% of cumulative population
polygon_ids_80pct = cum_pop.index[:idx_80]
print(f"{len(polygon_ids_80pct)=}")

cells_filter = gdf_combined_stats["polygon_id"].isin(polygon_ids_80pct)
print(f"{cells_filter.sum()=}")

centroids = gdf_combined_stats_polygon.centroid
base_group = centroids[~gdf_combined_stats_polygon.index.isin(polygon_ids_80pct)]
print(f"{base_group.shape=}")
group_80pct = centroids[gdf_combined_stats_polygon.index.isin(polygon_ids_80pct)]

pop_by_poly = gdf_combined_stats.groupby("polygon_id")["pop_primary_school_age"].sum()
base_group.plot(ax=ax, color="lightblue", markersize=2 * 10, alpha=0.2, zorder=4)
group_80pct.plot(
    ax=ax,
    color="blue",
    edgecolor="blue",
    linewidth=0.5,
    alpha=0.5,
    markersize=5 * 10,
    zorder=4,
)

###############

# Add a basemap with satellite imagery for context
# Set the map extent to the Amazon boundary before adding the basemap

ax.set_xlim(amazon_bounds[0], amazon_bounds[2])
ax.set_ylim(amazon_bounds[1], amazon_bounds[3])

ctx.add_basemap(
    ax,
    crs=countries.crs,
    # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
    source="https://a.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=10,
    attribution=False,
    attribution_size=0,
)
ctx.add_basemap(
    ax,
    crs=countries.crs,
    source="https://a.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=1,
    attribution=False,
    attribution_size=0,
)

# Add a legend for the Amazon boundary
# from matplotlib.lines import Line2D
# custom_lines = [Line2D([0], [0], color='red', lw=2.5)]
# ax.legend(custom_lines, ['Amazonia Boundary'], loc='lower left', fontsize=14, frameon=True)

# Remove axis ticks for a cleaner look
ax.set_xticks([])
ax.set_yticks([])

# Remove grid for a cleaner basemap
ax.grid(False)

# Set tight layout and show
# plt.tight_layout()
plt.savefig(
    "outputs/maps/amazon_cum_pop_80pct_primary.png",
    dpi=300,
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# Plot countries with soft color palette and nice borders
# Plot all countries in the Amazon with normal style
countries.clip(amazon).plot(
    ax=ax,
    column="ADM0_PCODE",
    # cmap="tab20b",
    # alpha=0.6,
    facecolor="none",
    edgecolor="k",
    linewidth=1.5,
    legend=False,
    zorder=2,
)
if not countries_no_data.empty:
    countries_no_data.plot(
        ax=ax,
        facecolor="none",
        edgecolor="k",
        linewidth=1.5,
        hatch="///",
        zorder=2,
    )

# Add crosshatch to Venezuela, Guyana, and Suriname to indicate no data
no_data_countries = ["VE", "GY", "SR"]
countries_no_data = countries[countries["ADM0_PCODE"].isin(no_data_countries)].clip(
    amazon
)

# Plot Amazon boundary with a bold red edge and no fill
amazon.boundary.plot(
    ax=ax, color="red", linewidth=1.5, zorder=3, label="Amazonia Boundary"
)


amazon_bounds = amazon.buffer(1).total_bounds
amazon_bbox_poly = gpd.GeoSeries(box(*amazon_bounds))
amazon_outer = amazon_bbox_poly.difference(amazon)
amazon_outer.plot(
    ax=ax, facecolor="k", edgecolor="none", linewidth=0, zorder=2, alpha=0.5
)

## DATA HERE ##

# Calculate cumulative population for each group
total_pop = middle_grouped[">30"].sum()
print(f"{total_pop.sum()=}")
cum_pop = middle_grouped[">30"].sort_values(ascending=False).cumsum()
idx_80 = (cum_pop.values >= 0.8 * total_pop).argmax()

# Get the polygon_ids that account for 80% of cumulative population
polygon_ids_80pct = cum_pop.index[:idx_80]
print(f"{len(polygon_ids_80pct)=}")

cells_filter = gdf_combined_stats["polygon_id"].isin(polygon_ids_80pct)
print(f"{cells_filter.sum()=}")

centroids = gdf_combined_stats_polygon.centroid
base_group = centroids[~gdf_combined_stats_polygon.index.isin(polygon_ids_80pct)]
print(f"{base_group.shape=}")
group_80pct = centroids[gdf_combined_stats_polygon.index.isin(polygon_ids_80pct)]

base_group.plot(ax=ax, color="moccasin", markersize=2 * 10, alpha=0.2, zorder=4)
group_80pct.plot(
    ax=ax,
    color="orange",
    edgecolor="orange",
    linewidth=0.5,
    alpha=0.5,
    markersize=5 * 10,
    zorder=4,
)

###############

# Add a basemap with satellite imagery for context
# Set the map extent to the Amazon boundary before adding the basemap

ax.set_xlim(amazon_bounds[0], amazon_bounds[2])
ax.set_ylim(amazon_bounds[1], amazon_bounds[3])

ctx.add_basemap(
    ax,
    crs=countries.crs,
    # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
    source="https://a.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=10,
    attribution=False,
    attribution_size=0,
)
ctx.add_basemap(
    ax,
    crs=countries.crs,
    source="https://a.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=1,
    attribution=False,
    attribution_size=0,
)

# Add a legend for the Amazon boundary
# from matplotlib.lines import Line2D
# custom_lines = [Line2D([0], [0], color='red', lw=2.5)]
# ax.legend(custom_lines, ['Amazonia Boundary'], loc='lower left', fontsize=14, frameon=True)

# Remove axis ticks for a cleaner look
ax.set_xticks([])
ax.set_yticks([])

# Remove grid for a cleaner basemap
ax.grid(False)

# Set tight layout and show
# plt.tight_layout()
plt.savefig(
    "outputs/maps/amazon_cum_pop_80pct_middle.png",
    dpi=300,
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# Plot countries with soft color palette and nice borders
# Plot all countries in the Amazon with normal style
countries.clip(amazon).plot(
    ax=ax,
    column="ADM0_PCODE",
    # cmap="tab20b",
    # alpha=0.6,
    facecolor="none",
    edgecolor="k",
    linewidth=1.5,
    legend=False,
    zorder=2,
)
if not countries_no_data.empty:
    countries_no_data.plot(
        ax=ax,
        facecolor="none",
        edgecolor="k",
        linewidth=1.5,
        hatch="///",
        zorder=2,
    )

# Add crosshatch to Venezuela, Guyana, and Suriname to indicate no data
no_data_countries = ["VE", "GY", "SR"]
countries_no_data = countries[countries["ADM0_PCODE"].isin(no_data_countries)].clip(
    amazon
)

# Plot Amazon boundary with a bold red edge and no fill
amazon.boundary.plot(
    ax=ax, color="red", linewidth=1.5, zorder=3, label="Amazonia Boundary"
)


amazon_bounds = amazon.buffer(1).total_bounds
amazon_bbox_poly = gpd.GeoSeries(box(*amazon_bounds))
amazon_outer = amazon_bbox_poly.difference(amazon)
amazon_outer.plot(
    ax=ax, facecolor="k", edgecolor="none", linewidth=0, zorder=2, alpha=0.5
)

## DATA HERE ##

# Calculate cumulative population for each group
total_pop = secondary_grouped[">30"].sum()
print(f"{total_pop.sum()=}")
cum_pop = secondary_grouped[">30"].sort_values(ascending=False).cumsum()
idx_80 = (cum_pop.values >= 0.8 * total_pop).argmax()

# Get the polygon_ids that account for 80% of cumulative population
polygon_ids_80pct = cum_pop.index[:idx_80]
print(f"{len(polygon_ids_80pct)=}")

centroids = gdf_combined_stats_polygon.centroid
base_group = centroids[~gdf_combined_stats_polygon.index.isin(polygon_ids_80pct)]
print(f"{base_group.shape=}")
group_80pct = centroids[gdf_combined_stats_polygon.index.isin(polygon_ids_80pct)]

base_group.plot(ax=ax, color="lightgreen", markersize=2 * 10, alpha=0.2, zorder=4)
group_80pct.plot(
    ax=ax,
    color="green",
    edgecolor="green",
    linewidth=0.5,
    alpha=0.5,
    markersize=5 * 10,
    zorder=4,
)

###############

# Add a basemap with satellite imagery for context
# Set the map extent to the Amazon boundary before adding the basemap

ax.set_xlim(amazon_bounds[0], amazon_bounds[2])
ax.set_ylim(amazon_bounds[1], amazon_bounds[3])

ctx.add_basemap(
    ax,
    crs=countries.crs,
    # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
    source="https://a.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=10,
    attribution=False,
    attribution_size=0,
)
ctx.add_basemap(
    ax,
    crs=countries.crs,
    source="https://a.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=1,
    attribution=False,
    attribution_size=0,
)

# Add a legend for the Amazon boundary
# from matplotlib.lines import Line2D
# custom_lines = [Line2D([0], [0], color='red', lw=2.5)]
# ax.legend(custom_lines, ['Amazonia Boundary'], loc='lower left', fontsize=14, frameon=True)

# Remove axis ticks for a cleaner look
ax.set_xticks([])
ax.set_yticks([])

# Remove grid for a cleaner basemap
ax.grid(False)

# Set tight layout and show
# plt.tight_layout()
plt.savefig(
    "outputs/maps/amazon_cum_pop_80pct_secondary.png",
    dpi=300,
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()

In [ ]:
# Genial Clau. Priorizaria 3 5 y 6, y que en el 3 tengamos los asentamientos también delimitados.

In [ ]:
# Mapa 1. Region de interés, mapa satelital con países por colores con transparencia y la zona de la Amazonía con borde rojo

# Mapa 2. Mapa de población de referencia (WorldPop a 100m2 o 1km2 dependiendo como se vea mejor) de la Amazonia

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
from pyproj import Transformer
from shapely.geometry import box

In [ ]:
# Read the country polygons
countries = gpd.read_parquet(
    "~/Documents/amazonia-bid/outputs/amazon_countries.parquet"
)
# Load amazon boundaries
amazon = gpd.read_file("/Users/claudio/Documents/amazonia-bid/inputs/Amazonas")
amazon.crs.to_string()

In [ ]:
amazon = amazon.to_crs(countries.crs)

In [ ]:
# Create a beautiful basemap for the next maps (heatmaps, etc)
fig, ax = plt.subplots(figsize=(12, 12))

# Plot countries with soft color palette and nice borders
countries.clip(amazon).plot(
    ax=ax,
    column="ADM0_PCODE",
    # cmap="tab20b",
    # alpha=0.6,
    facecolor="none",
    edgecolor="k",
    linewidth=1.5,
    legend=False,
    zorder=2,
)

# Plot Amazon boundary with a bold red edge and no fill
amazon.boundary.plot(
    ax=ax, color="red", linewidth=2.5, zorder=3, label="Amazonia Boundary"
)

# Add crosshatch to Venezuela, Guyana, and Suriname to indicate no data
no_data_countries = ["VE", "GY", "SR"]
countries_no_data = countries[countries["ADM0_PCODE"].isin(no_data_countries)].clip(
    amazon
)
if not countries_no_data.empty:
    countries_no_data.plot(
        ax=ax,
        facecolor="none",
        edgecolor="k",
        linewidth=1.5,
        hatch="///",
        zorder=2,
    )


amazon_bounds = amazon.buffer(1).total_bounds
amazon_bbox_poly = gpd.GeoSeries(box(*amazon_bounds))
amazon_outer = amazon_bbox_poly.difference(amazon)
amazon_outer.plot(
    ax=ax, facecolor="k", edgecolor="none", linewidth=0, zorder=2, alpha=0.5
)

# Add a basemap with satellite imagery for context
# Set the map extent to the Amazon boundary before adding the basemap

ax.set_xlim(amazon_bounds[0], amazon_bounds[2])
ax.set_ylim(amazon_bounds[1], amazon_bounds[3])

ctx.add_basemap(
    ax,
    source=ctx.providers.Esri.WorldImagery,
    crs=countries.crs,
    alpha=0.8,
    zorder=1,
    attribution_size=0,
    attribution=False,
)

# Add a semi-transparent white overlay to soften the satellite imagery
# ax.patch.set_alpha(0.7)

# Add a legend for the Amazon boundary
# from matplotlib.lines import Line2D
# custom_lines = [Line2D([0], [0], color='red', lw=2.5)]
# ax.legend(custom_lines, ['Amazonia Boundary'], loc='lower left', fontsize=14, frameon=True)

# Remove axis ticks for a cleaner look
ax.set_xticks([])
ax.set_yticks([])

# Add a beautiful title
# ax.set_title(
#     "Amazon Basin and Country Boundaries", fontsize=20, fontweight="bold", pad=20
# )

# Remove axis
ax.set_axis_off()

# Set tight layout and show
# plt.tight_layout()
plt.savefig(
    "outputs/maps/01_region_of_analysis.png",
    dpi=300,
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()

In [ ]:
import urbanpy as up

In [ ]:
amazon_hexs = up.geom.gen_hexagons(resolution=5, city=amazon)

In [ ]:
gdf_schools = gpd.read_parquet("outputs/amazon_schools.parquet")
print(f"{gdf_schools.shape[0]} Cells loaded")

In [ ]:
gdf_schools.id_edificio.unique().shape[0] == gdf_schools.shape[0]

In [ ]:
amazon_hexs_schools_counts = up.geom.merge_shape_hex(
    hexs=amazon_hexs,
    shape=gdf_schools,
    how="left",
    predicate="within",
    agg={"id_edificio": "count"},
)

In [ ]:
amazon_hexs_schools_counts_res4 = up.geom.resolution_downsampling(
    gdf=amazon_hexs_schools_counts,
    hex_col="hex",
    coarse_resolution=4,
    agg={"id_edificio": "sum"},
)

In [ ]:
amazon_hexs_schools_counts.query("id_edificio > 0").plot(
    "id_edificio",
    figsize=(10, 10),
    edgecolor="none",
    linewidth=0,
    cmap="viridis",
    legend=True,
    legend_kwds={"label": "Número de escuelas"},
)

plt.show()

In [ ]:
amazon_hexs_schools_counts_res4.query("id_edificio > 0").plot(
    "id_edificio",
    figsize=(10, 10),
    edgecolor="none",
    linewidth=0,
    cmap="viridis",
    legend=True,
    legend_kwds={"label": "Número de escuelas"},
)

plt.show()

In [ ]:
countries["ADM0_PCODE"]

In [ ]:
# Mapa 3. Mapa de concentración de escuelas heatmap

# Add colorbar with custom positioning
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Create a beautiful basemap for the next maps (heatmaps, etc)
fig, ax = plt.subplots(figsize=(12, 12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Plot countries with soft color palette and nice borders
# Plot all countries in the Amazon with normal style
countries.clip(amazon).plot(
    ax=ax,
    column="ADM0_PCODE",
    # cmap="tab20b",
    # alpha=0.6,
    facecolor="none",
    edgecolor="k",
    linewidth=1.5,
    legend=False,
    zorder=2,
)

# Add crosshatch to Venezuela, Guyana, and Suriname to indicate no data
no_data_countries = ["VE", "GY", "SR"]
countries_no_data = countries[countries["ADM0_PCODE"].isin(no_data_countries)].clip(
    amazon
)
if not countries_no_data.empty:
    countries_no_data.plot(
        ax=ax,
        facecolor="none",
        edgecolor="k",
        linewidth=1.5,
        hatch="///",
        zorder=2,
    )

# Plot Amazon boundary with a bold red edge and no fill
amazon.boundary.plot(
    ax=ax, color="red", linewidth=2.5, zorder=3, label="Amazonia Boundary"
)


amazon_bounds = amazon.buffer(1).total_bounds
amazon_bbox_poly = gpd.GeoSeries(box(*amazon_bounds))
amazon_outer = amazon_bbox_poly.difference(amazon)
amazon_outer.plot(
    ax=ax, facecolor="k", edgecolor="none", linewidth=0, zorder=2, alpha=0.5
)


## DATA HERE ##

amazon_hexs_schools_counts_res4.query("id_edificio > 0").plot(
    "id_edificio",
    edgecolor="none",
    linewidth=0,
    cmap="viridis",
    legend=True,
    legend_kwds={"label": "Número de escuelas", "ticks": [0, 100, 200, 300, 400]},
    alpha=0.5,
    ax=ax,
    cax=cax,
    zorder=4,
    vmax=400,
)


###############

# Add a basemap with satellite imagery for context
# Set the map extent to the Amazon boundary before adding the basemap

ax.set_xlim(amazon_bounds[0], amazon_bounds[2])
ax.set_ylim(amazon_bounds[1], amazon_bounds[3])

ctx.add_basemap(
    ax,
    crs=countries.crs,
    # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
    source="https://a.basemaps.cartocdn.com/dark_only_labels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=10,
    attribution=False,
    attribution_size=0,
)
ctx.add_basemap(
    ax,
    crs=countries.crs,
    source=ctx.providers.Esri.WorldImagery,
    alpha=1.0,
    zorder=1,
    attribution=False,
    attribution_size=0,
)

# Add a legend for the Amazon boundary
# from matplotlib.lines import Line2D
# custom_lines = [Line2D([0], [0], color='red', lw=2.5)]
# ax.legend(custom_lines, ['Amazonia Boundary'], loc='lower left', fontsize=14, frameon=True)

# Remove axis ticks for a cleaner look
ax.set_xticks([])
ax.set_yticks([])

# Remove grid for a cleaner basemap
ax.grid(False)

# Set tight layout and show
# plt.tight_layout()
plt.savefig(
    "outputs/maps/amazon_schools_heatmap_res4.png", dpi=300, bbox_inches="tight"
)
plt.show()

In [ ]:
from osmnx import project_gdf

In [ ]:
from shapely import Point, box

BUFFER_1KM = 0.009 / 2
BUFFER_5KM = BUFFER_1KM * 5
BUFFER_10KM = BUFFER_1KM * 10
BUFFER_25KM = BUFFER_1KM * 25
kmbox = gpd.GeoSeries([box(*Point(-60, -3.1).buffer(BUFFER_25KM).bounds)], crs=4326)
project_gdf(kmbox).area / 1e6  # area in km^2

In [ ]:
# Create a beautiful basemap for the next maps (heatmaps, etc)
fig, ax = plt.subplots(figsize=(12, 12))

# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.1)

# plot the raster data for brazil
xds_band.rio.clip([kmbox.geometry.iloc[0]]).plot.imshow(
    ax=ax,
    cmap="viridis",
    alpha=0.5,
    add_colorbar=False,
    # cbar_ax=cax
)

# plot the sample cell
# kmbox.plot(ax=ax, facecolor="none", edgecolor="white", linewidth=1, alpha=1)
# kmbox.centroid.plot(ax=ax, color="red", linewidth=1, alpha=1)

# plot the subregion
# xds_clipped.plot.imshow(ax=ax, cmap="Reds", alpha=0.75, add_colorbar=False)

# set the x and y limits to the sample cell buffer

ax.set_xlim(kmbox.total_bounds[0], kmbox.total_bounds[2])
ax.set_ylim(kmbox.total_bounds[1], kmbox.total_bounds[3])

# Add a basemap below all other layers
ctx.add_basemap(
    ax,
    crs=gdf_celdas_bra_clean.crs,
    # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
    source="https://a.basemaps.cartocdn.com/dark_only_labels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=0,
)

ctx.add_basemap(
    ax,
    crs=gdf_celdas_bra_clean.crs,
    # source=ctx.providers.Esri.WorldImagery,
    source="https://a.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=-2,
    attribution=False,
    attribution_size=0,
)

ax.set_title(None)
ax.set_axis_off()

# Set tight layout and show
# plt.tight_layout()
plt.savefig(
    "outputs/maps/02_woldpop_sample.png", dpi=300, bbox_inches="tight", pad_inches=0
)
plt.show()

In [ ]:
# Mapa 4. Mapa de ríos en la Amazonía

# Mapa 5. Mapa 3D zoom a asentamiento muestra población escolar(celda),  escuelas (punto) y arco (color según cat de tiempo de viaje)

# Mapa 6. Igual que 5 pero un caso de ribereña

In [ ]:
urban_area_duration_primary = gdf_combined_stats[
    gdf_combined_stats["category"] == "urban_area"
]["duration_to_nearest_primary_schools"].describe()
non_urban_area_duration_primary = gdf_combined_stats[
    gdf_combined_stats["category"] == "non_urban_area"
]["duration_to_nearest_primary_schools"].describe()

# plot the diference between urban and non-urban areas on the mean and median of the duration to nearest primary school
plt.figure(figsize=(10, 6))
plt.bar(
    ["Urban Area", "Non-Urban Area"],
    [urban_area_duration_primary["mean"], non_urban_area_duration_primary["mean"]],
    capsize=5,
    color=["blue", "orange"],
)
plt.ylabel("Mean Duration to Nearest Primary School (minutes)")
plt.title("Mean Duration to Nearest Primary School by Area Type")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# plot the diference between urban and non-urban areas on the median of the duration to nearest primary school
plt.figure(figsize=(10, 6))
plt.bar(
    ["Urban Area", "Non-Urban Area"],
    [urban_area_duration_primary["50%"], non_urban_area_duration_primary["50%"]],
    capsize=5,
    color=["blue", "orange"],
)
plt.ylabel("Median Duration to Nearest Primary School (minutes)")
plt.title("Median Duration to Nearest Primary School by Area Type")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
gdf_combined_stats[gdf_combined_stats["category"] == "urban_area"][
    "duration_to_nearest_primary_schools"
].describe()

In [ ]:
gdf_combined_stats["duration_to_nearest_primary_schools"].unique()

In [ ]:
gdf_combined_stats["duration_to_nearest_primary_schools"].isna().sum()

In [ ]:
(gdf_combined_stats["duration_to_nearest_primary_schools"] == 0).sum()

In [ ]:
gdf_combined_stats[gdf_combined_stats["duration_to_nearest_primary_schools"] > 0][
    "duration_to_nearest_primary_schools"
].describe()

In [ ]:
ax = gdf_combined_stats[gdf_combined_stats["duration_to_nearest_primary_schools"] > 0][
    "duration_to_nearest_primary_schools"
].hist(bins=1000)
ax.set_xlim(0, 100)

for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.annotate(
            f"{int(height)}",
            (p.get_x() + p.get_width() / 2, height),
            ha="center",
            va="bottom",
            fontsize=8,
            rotation=0,
            xytext=(0, 3),
            textcoords="offset points",
        )

plt.xlabel("Duration to Nearest Primary School (minutes)")
plt.ylabel("Frequency")
plt.title("Distribution of Duration to Nearest Primary School")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
ax = gdf_combined_stats["duration_to_nearest_primary_schools"].hist(bins=1000)
ax.set_xlim(0, 100)

for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.annotate(
            f"{int(height)}",
            (p.get_x() + p.get_width() / 2, height),
            ha="center",
            va="bottom",
            fontsize=8,
            rotation=0,
            xytext=(0, 3),
            textcoords="offset points",
        )

plt.xlabel("Duration to Nearest Primary School (minutes)")
plt.ylabel("Frequency")
plt.title("Distribution of Duration to Nearest Primary School")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
duration_bins = [0, 5, 10, 15, 20, 30, 45, 60, 90, 120, 180, 240, 300, 360, 420, 480]

In [ ]:
plt.figure(figsize=(15, 8))
sns.histplot(
    data=gdf_combined_stats,
    x="duration_to_nearest_primary_schools",
    hue="category",
    bins=duration_bins,
    element="step",
    stat="count",
    common_norm=False,
    alpha=0.7,
)
plt.title("Duration to Nearest Primary School by Area Type")
plt.xlabel("Duration to Nearest Primary School (minutes)")
plt.ylabel("Cell Count")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 12), sharey="row")
for i, level in enumerate(["Primary", "Middle", "Secondary"]):
    df_plot = summary_df[summary_df["education_level"] == level]
    sns.pointplot(
        data=df_plot,
        x="total_pop",
        y="country",
        hue="category",
        legend=i == 0,
        ax=axes[i, 0],
        palette="Set2",
        join=False,
        alpha=0.5,
        markers="o",
    )
    # Connect dots from the same country but different hue (category)
    for country, group in df_plot.groupby("country"):
        # Sort by hue order if needed
        group_sorted = group.sort_values("category")
        axes[i, 0].plot(
            group_sorted["total_pop"],
            group_sorted["country"],
            color="gray",
            linewidth=1,
            alpha=0.7,
            zorder=0,
        )
    axes[i, 0].set_title(f"{level}: Total Population")
    axes[i, 0].set_xlabel("Population")
    axes[i, 0].set_ylabel("Country")

    sns.pointplot(
        data=df_plot,
        x="mean_duration",
        y="country",
        hue="category",
        legend=False,
        ax=axes[i, 1],
        palette="Set2",
        join=False,
        # dodge=0.3,
        alpha=0.5,
        markers="o",
    )
    for country, group in df_plot.groupby("country"):
        # Sort by hue order if needed
        group_sorted = group.sort_values("category")
        axes[i, 1].plot(
            group_sorted["mean_duration"],
            group_sorted["country"],
            color="gray",
            linewidth=1,
            alpha=0.7,
            zorder=0,
        )
    axes[i, 1].set_title(f"{level}: Mean Travel Time (min)")
    axes[i, 1].set_xlabel("Mean Duration")
    axes[i, 1].set_ylabel("Country")

    sns.pointplot(
        data=df_plot,
        x="median_duration",
        y="country",
        hue="category",
        legend=False,
        ax=axes[i, 2],
        palette="Set2",
        join=False,
        # dodge=0.3,
        alpha=0.5,
        markers="o",
    )
    for country, group in df_plot.groupby("country"):
        # Sort by hue order if needed
        group_sorted = group.sort_values("category")
        axes[i, 2].plot(
            group_sorted["median_duration"],
            group_sorted["country"],
            color="gray",
            linewidth=1,
            alpha=0.7,
            zorder=0,
        )
    axes[i, 2].set_title(f"{level}: Median Travel Time (min)")
    axes[i, 2].set_xlabel("Median Duration")
    axes[i, 2].set_ylabel("Country")

plt.tight_layout()
plt.show()

In [ ]:
# Get the top 5 polygons (human settlements) by population that are more than 30 minutes away from the closest school for each education level.
top_5_primary = primary_grouped[">30"].nlargest(5)
top_5_secondary = secondary_grouped[">30"].nlargest(5)
top_5_middle = middle_grouped[">30"].nlargest(5)

In [ ]:
from pathlib import Path

In [ ]:
# We'll create one map for each polygon_id, education level, and travel time category.
for top_5, edu_level, label, color in [
    (top_5_primary, "nivel_primaria", "primary", "blue"),
    (top_5_middle, "nivel_media", "middle", "orange"),
    (top_5_secondary, "nivel_secundaria", "secondary", "green"),
]:
    for poly_id in top_5.index:

        plot_gdf = gdf_combined_stats.query("polygon_id == @poly_id")
        gdf_schools_in_poly = gdf_schools.cx[
            plot_gdf.total_bounds[0] : plot_gdf.total_bounds[2],
            plot_gdf.total_bounds[1] : plot_gdf.total_bounds[3],
        ]
        # print(gdf_schools_in_poly)

        ## START PLOT ##
        fig, ax = plt.subplots(figsize=(10, 10))

        schools_to_plot = gdf_schools_in_poly[gdf_schools_in_poly[edu_level] == 1]

        if not schools_to_plot.empty:
            schools_to_plot.plot(
                ax=ax,
                color=color,
                markersize=10,
                alpha=0.5,
                zorder=3,
            )
        else:
            print(f"No schools found for {edu_level} in polygon {poly_id}")

        ## plot Travel Time Categories
        plot_gdf.plot(
            f"duration_to_nearest_{label}_schools_label",
            linewidth=0,
            alpha=0.7,
            cmap="RdYlGn_r",
            legend=True,
            ax=ax,
            zorder=3,
        )

        ax.set_axis_off()
        # Add carto light basemap only labels
        ctx.add_basemap(
            ax,
            crs=plot_gdf.crs,
            source="https://{s}.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}{r}@2x.png",
            alpha=1.0,
            zorder=10,
            attribution=False,
            attribution_size=0,
        )
        ctx.add_basemap(
            ax,
            crs=plot_gdf.crs,
            source="https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}@2x.png",
            alpha=1.0,
            zorder=1,
            attribution=False,
            attribution_size=0,
        )
        filename = Path(
            f"outputs/maps/top_5_settlements_by_travel_time_and_pop/{label}/{poly_id}.png"
        )
        # create parent
        filename.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(
            filename,
            dpi=300,
            bbox_inches="tight",
            pad_inches=0,
        )
        plt.show()

In [ ]:
def plot_settlement_access(poly_id, filename, edu_level, label, color):
    plot_gdf = gdf_combined_stats.query("polygon_id == @poly_id")
    gdf_schools_in_poly = gdf_schools.cx[
        plot_gdf.total_bounds[0] : plot_gdf.total_bounds[2],
        plot_gdf.total_bounds[1] : plot_gdf.total_bounds[3],
    ]

    fig, ax = plt.subplots(figsize=(10, 10))

    schools_to_plot = gdf_schools_in_poly[gdf_schools_in_poly[edu_level] == 1]

    if not schools_to_plot.empty:
        schools_to_plot.plot(
            ax=ax,
            color=color,
            markersize=10,
            alpha=0.5,
            zorder=3,
        )
    else:
        print(f"No schools found for {edu_level} in polygon {poly_id}")

    plot_gdf.plot(
        f"duration_to_nearest_{label}_schools_label",
        linewidth=0,
        alpha=0.7,
        cmap="RdYlGn_r",
        legend=True,
        legend_kwds={
            "loc": "upper right",
            "fontsize": "xx-small",
        },
        ax=ax,
        zorder=3,
    )

    ax.set_axis_off()
    ctx.add_basemap(
        ax,
        crs=plot_gdf.crs,
        source="https://{s}.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}{r}@2x.png",
        alpha=1.0,
        zorder=4,
        attribution=False,
        attribution_size=0,
    )
    ctx.add_basemap(
        ax,
        crs=plot_gdf.crs,
        source="https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}@2x.png",
        alpha=1.0,
        zorder=1,
        attribution=False,
        attribution_size=0,
    )
    filename = Path(filename)
    filename.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(
        filename,
        dpi=300,
        bbox_inches="tight",
        pad_inches=0,
    )
    plt.show()

In [ ]:
manaus_polygon_id = "01810-1-3"
plot_settlement_access(
    manaus_polygon_id,
    "outputs/maps/manaus_primary_schools_and_travel_time_categories.png",
    "nivel_primaria",
    "primary",
    "blue",
)
# plot_settlement_access(manaus_polygon_id, "outputs/maps/manaus_primary_schools_and_travel_time_categories.png", "nivel_media", "middle", "orange")
# plot_settlement_access(manaus_polygon_id, "outputs/maps/manaus_primary_schools_and_travel_time_categories.png", "nivel_secundaria", "secondary", "green")

In [ ]:
# Mapa 5. Mapa 3D zoom a asentamiento muestra población escolar(celda),
# escuelas (punto) y arco (color según cat de tiempo de viaje)

top5_polygons_pop_worst_access = (
    primary_grouped[">30"].sort_values(ascending=False).head(5)
)

In [ ]:
for poly_id in top5_polygons_pop_worst_access.index:

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    plot_gdf = gdf_combined_stats.query("polygon_id == @poly_id")
    # print(plot_gdf.shape)
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    # Create a divider for the existing axes
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Plot the data without the legend
    pop_plot = plot_gdf.plot(
        "pop_primary_school_age",
        linewidth=0,
        alpha=0.25,
        legend=True,
        ax=ax,
        cax=cax,
        cmap="Reds",
    )

    # Plot and get the mappable for the legend
    # Use an axes divider for the duration labels legend
    # duration_divider = make_axes_locatable(ax)
    # duration_cax = duration_divider.append_axes("bottom", size="5%", pad=0.5)

    mappable = plot_gdf.plot(
        "duration_to_nearest_primary_schools_label",
        linewidth=1,
        facecolor="none",
        legend=True,
        legend_kwds={
            "loc": "lower left",
            "bbox_to_anchor": (0.0, -0.1),
            "ncols": 8,
            "fontsize": "xx-small",
            # "mode": "expand",
        },
        ax=ax,
        # legend_ax= duration_cax,
        cmap="RdYlGn_r",
    )
    # Filter school in the total bounds of plot_gdf
    plot_gdf_bounds = plot_gdf.total_bounds
    schools_in_poly = gdf_schools.cx[
        plot_gdf_bounds[0] : plot_gdf_bounds[2],
        plot_gdf_bounds[1] : plot_gdf_bounds[3],
    ]
    # schools_in_poly = schools_in_poly.query("id_edificio > 0")
    # print(schools_in_poly.shape)
    schools_in_poly.plot(ax=ax, color="green", markersize=10, alpha=0.5)

    ctx.add_basemap(
        ax,
        source=ctx.providers.Esri.WorldImagery,
        crs=plot_gdf.crs,
        attribution=False,
        attribution_size=0,
    )
    ctx.add_basemap(
        ax,
        crs=countries.crs,
        # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
        source="https://a.basemaps.cartocdn.com/dark_only_labels/{z}/{x}/{y}@2x.png",
        alpha=1.0,
        zorder=10,
        attribution=False,
        attribution_size=0,
    )

    # plt.suptitle(f"polygon_id={poly_id}")

    ax.set_axis_off()

    # plt.tight_layout()

    plt.show()

In [ ]:
primary_grouped[primary_grouped["No access"].sort_values(ascending=False) > 1].shape

In [ ]:
top5_polygons_pop_no_access = (
    primary_grouped["No access"].sort_values(ascending=False).head(5)
)
top5_polygons_pop_no_access

In [ ]:
for poly_id in top5_polygons_pop_no_access.index:

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    plot_gdf = gdf_combined_stats.query("polygon_id == @poly_id")
    # print(plot_gdf.shape)
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    # Create a divider for the existing axes
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Plot the data without the legend
    pop_plot = plot_gdf.plot(
        "pop_primary_school_age",
        linewidth=0,
        alpha=0.25,
        legend=True,
        ax=ax,
        cax=cax,
        cmap="Reds",
    )

    # Plot and get the mappable for the legend
    # Use an axes divider for the duration labels legend
    # duration_divider = make_axes_locatable(ax)
    # duration_cax = duration_divider.append_axes("bottom", size="5%", pad=0.5)

    mappable = plot_gdf.plot(
        "duration_to_nearest_primary_schools_label",
        linewidth=1,
        facecolor="none",
        legend=True,
        legend_kwds={
            "loc": "lower left",
            "bbox_to_anchor": (0.0, -0.1),
            "ncols": 8,
            "fontsize": "xx-small",
            # "mode": "expand",
        },
        ax=ax,
        # legend_ax= duration_cax,
        cmap="RdYlGn_r",
    )
    # Filter school in the total bounds of plot_gdf
    plot_gdf_bounds = plot_gdf.total_bounds
    schools_in_poly = gdf_schools.cx[
        plot_gdf_bounds[0] : plot_gdf_bounds[2],
        plot_gdf_bounds[1] : plot_gdf_bounds[3],
    ]
    # schools_in_poly = schools_in_poly.query("id_edificio > 0")
    # print(schools_in_poly.shape)
    # schools_in_poly.plot(ax=ax, color="green", markersize=10, alpha=0.5)
    schools_in_poly.plot(
        ax=ax,
        column="nivel_primaria",
        markersize=10,
        alpha=0.5,
        legend=True,
        categorical=True,
        cmap="viridis",
    )

    ctx.add_basemap(
        ax,
        source=ctx.providers.Esri.WorldImagery,
        crs=plot_gdf.crs,
        attribution=False,
        attribution_size=0,
    )
    ctx.add_basemap(
        ax,
        crs=countries.crs,
        # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
        source="https://a.basemaps.cartocdn.com/dark_only_labels/{z}/{x}/{y}@2x.png",
        alpha=1.0,
        zorder=10,
        attribution=False,
        attribution_size=0,
    )

    # plt.suptitle(f"polygon_id={poly_id}")

    ax.set_axis_off()

    # plt.tight_layout()

    plt.show()

In [ ]:
connected_riverside_regions = gpd.read_file(
    "~/Downloads/ROTA ESCOLAR RIBEIRINHA.kml", layer="REGIÕES DE ROTAS CONECTADAS"
)

routes = gpd.read_file(
    "~/Downloads/ROTA ESCOLAR RIBEIRINHA.kml", layer="ROTA", on_invalid="ignore"
)

In [ ]:
routes.plot()

In [ ]:
connected_riverside_regions.union_all()

In [ ]:
connected_riverside_regions.total_bounds

In [ ]:
connected_riverside_regions

In [ ]:
cells_in_conn_regions = gdf_combined_stats.cx[
    connected_riverside_regions.total_bounds[
        0
    ] : connected_riverside_regions.total_bounds[2],
    connected_riverside_regions.total_bounds[
        1
    ] : connected_riverside_regions.total_bounds[3],
]

In [ ]:
connected_riverside_regions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Create a divider for the existing axes
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

# Plot the data without the legend
cells_in_conn_regions.plot(
    "pop_primary_school_age",
    linewidth=0,
    alpha=0.25,
    legend=True,
    ax=ax,
    cax=cax,
    cmap="Reds",
)

# Plot and get the mappable for the legend
# Use an axes divider for the duration labels legend
# duration_divider = make_axes_locatable(ax)
# duration_cax = duration_divider.append_axes("bottom", size="5%", pad=0.5)

cells_in_conn_regions.plot(
    "duration_to_nearest_primary_schools_label",
    linewidth=1,
    facecolor="none",
    legend=True,
    legend_kwds={
        "loc": "lower left",
        "bbox_to_anchor": (0.0, -0.1),
        "ncols": 8,
        "fontsize": "xx-small",
        # "mode": "expand",
    },
    ax=ax,
    # legend_ax= duration_cax,
    cmap="RdYlGn_r",
)
# Filter school in the total bounds of plot_gdf
plot_gdf_bounds = cells_in_conn_regions.total_bounds
schools_in_poly = gdf_schools.cx[
    plot_gdf_bounds[0] : plot_gdf_bounds[2],
    plot_gdf_bounds[1] : plot_gdf_bounds[3],
]
# schools_in_poly = schools_in_poly.query("id_edificio > 0")
# print(schools_in_poly.shape)
# schools_in_poly.plot(ax=ax, color="green", markersize=10, alpha=0.5)
schools_in_poly.plot(
    ax=ax,
    column="nivel_primaria",
    markersize=10,
    alpha=0.5,
    legend=True,
    categorical=True,
    cmap="viridis",
)

connected_riverside_regions.plot(
    linewidth=2, facecolor="none", ax=ax, edgecolor="orange"
)

ctx.add_basemap(
    ax,
    source=ctx.providers.Esri.WorldImagery,
    crs=plot_gdf.crs,
    attribution=False,
    attribution_size=0,
)
ctx.add_basemap(
    ax,
    crs=countries.crs,
    # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
    source="https://a.basemaps.cartocdn.com/dark_only_labels/{z}/{x}/{y}@2x.png",
    alpha=1.0,
    zorder=10,
    attribution=False,
    attribution_size=0,
)

# plt.suptitle(f"polygon_id={poly_id}")

ax.set_axis_off()

# plt.tight_layout()

plt.show()

In [ ]:
m = cells_in_conn_regions.explore(
    "pop_primary_school_age",
    tiles=ctx.providers.Esri.WorldImagery,
    cmap="Reds",
    style_kwds={"stroke": False},
)
cells_in_conn_regions.explore(
    "duration_to_nearest_primary_schools_label",
    m=m,
    cmap="RdYlGn_r",
    style_kwds={"fill": False},
)
schools_in_poly.explore(
    m=m,
    column="nivel_primaria",
    categorical=True,
    cmap="viridis",
)
connected_riverside_regions.explore(m=m, style_kwds={"fill": False})
routes.explore(m=m, style_kwds={"fill": False}, color="blue")


display(m)

In [ ]:
top5_polygons_pop_no_access

In [ ]:
for poly_id in top5_polygons_pop_no_access.index:

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    plot_gdf = gdf_combined_stats.query("polygon_id == @poly_id")
    # print(plot_gdf.shape)
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    # Create a divider for the existing axes
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Plot the data without the legend
    pop_plot = plot_gdf.plot(
        "pop_primary_school_age",
        linewidth=0,
        alpha=0.25,
        legend=True,
        ax=ax,
        cax=cax,
        cmap="Reds",
    )

    # Plot and get the mappable for the legend
    # Use an axes divider for the duration labels legend
    # duration_divider = make_axes_locatable(ax)
    # duration_cax = duration_divider.append_axes("bottom", size="5%", pad=0.5)

    mappable = plot_gdf.plot(
        "duration_to_nearest_primary_schools_label",
        linewidth=1,
        facecolor="none",
        legend=True,
        legend_kwds={
            "loc": "lower left",
            "bbox_to_anchor": (0.0, -0.1),
            "ncols": 8,
            "fontsize": "xx-small",
            # "mode": "expand",
        },
        ax=ax,
        # legend_ax= duration_cax,
        cmap="RdYlGn_r",
    )
    # Filter school in the total bounds of plot_gdf
    plot_gdf_bounds = plot_gdf.total_bounds
    schools_in_poly = gdf_schools.cx[
        plot_gdf_bounds[0] : plot_gdf_bounds[2],
        plot_gdf_bounds[1] : plot_gdf_bounds[3],
    ]
    # schools_in_poly = schools_in_poly.query("id_edificio > 0")
    # print(schools_in_poly.shape)
    # schools_in_poly.plot(ax=ax, color="green", markersize=10, alpha=0.5)
    schools_in_poly.plot(
        ax=ax,
        column="nivel_primaria",
        markersize=10,
        alpha=0.5,
        legend=True,
        categorical=True,
        cmap="viridis",
    )

    ctx.add_basemap(
        ax,
        source=ctx.providers.Esri.WorldImagery,
        crs=plot_gdf.crs,
        attribution=False,
        attribution_size=0,
    )
    ctx.add_basemap(
        ax,
        crs=countries.crs,
        # source=ctx.providers.CartoDB.DarkMatterOnlyLabels,
        source="https://a.basemaps.cartocdn.com/dark_only_labels/{z}/{x}/{y}@2x.png",
        alpha=1.0,
        zorder=10,
        attribution=False,
        attribution_size=0,
    )

    # plt.suptitle(f"polygon_id={poly_id}")

    ax.set_axis_off()

    # plt.tight_layout()

    plt.show()